# DeepAR Notes and Questions

Do we need to hot encode city? Wouldn't latitude and longitude be better data in terms of percentages of fire within a given location?

If cities were given a binary value, wouldn't the predictions be more suscetible to prediciting fires ONLY in the cities that had fires previously, and give a 0 percent prediction on cities that weren't in our training data?

But I guess if the user is giving us a location, hot encoding the locations would be better.

BUT.... web-services could take the location, web-scrape its latitude and longitude and have them pass those values to us to make a prediction.

Just thinking out loud, maybe something we could consider.

## So changing the algorithm to xgboost because DeepAR only works with a date and a list of target attributes
## XGBoost is better in finding a continuous value given a multitude of attribute data

We will need to convert JSON into csvs.... hope that's not too much trouble

# Setup
Importing Sagemaker and setting file parameters. DO NOT RUN THESE UNTIL WE ARE READY TO SEND THIS NOTEBOOK INTO AWS SAGEMAKER!

In [20]:
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
prefix = "sagemaker/wildfire-xgboost"

# Define IAM role
import boto3
import re
from sagemaker import get_execution_role

role = get_execution_role()

Import Necessary Files and packages

In [21]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import io
import os
import sys
import time
import json
from IPython.display import display
from time import strftime, gmtime
from sagemaker.inputs import TrainingInput
from sagemaker.serializers import CSVSerializer

# Setting Up Data
Download the cleaned CSV file here

Refer to tutorial for assistance

Mak will implmenet the preparation of the data by adding non"true" fire results to the cleaned csv.

To accomplish this, we need to loop through every fire or weather station
for each weather station, we loop throughout the range of dates... yes every single one, and add a 1 to 
the target attribute if there was a fire on that specific day, 0 if otherwise

This will allow the DeepAR learning algorithm to gain a probability of a fire in a particular city.

NOTE, this might prove challenging if the input for location is converted to latitude and longitude but im certain 
it's doable

In [1]:
# Download the cleaned CSV file here
# Refer to tutorial for assistance
# Mak will implmenet the preparation of the data by adding non"true" fire results to the cleaned csv
# To accomplish this, we need to loop through every fire or weather station
# For each weather station, we loop throughout the range of dates... yes every single one, and add a 1 to 
# the target attribute if there was a fire on that specific day, 0 if otherwise
# This will allow the DeepAR learning algorithm to gain a probability of a fire in a particular city.
# NOTE, this might prove challenging if the input for location is converted to latitude and longitude but im certain 
# it's doable

# So changing the algorithm to xgboost because DeepAR only works with a date and a list of target attributes
# XGBoost is better in finding a continuous value given a multitude of attribute data

In [22]:
# download the data
s3 = boto3.client("s3")
datafile = "fireWithWeather.csv"

df = pd.read_csv(datafile)

fire_data = df[:33]


fire_data


# Hopefully it's clean

,Fire,Latitude,Longitude,Started,Precipitation30,Precipitation60,Precipitation90,Humidity,Temperature
0,1,37.857000,-120.086000,2013-08-17 15:25:00,0.000000,0.000167,0.001333,39.0,79.8
1,1,34.585595,-118.423176,2013-05-30 15:28:00,0.009333,0.004667,0.010556,39.0,72.8
2,1,33.709500,-116.728850,2013-07-15 13:43:00,0.000000,0.000000,0.000000,23.0,94.0
3,1,33.861570,-116.904270,2013-08-07 14:05:00,0.005333,0.009167,0.010444,39.0,72.4
4,1,40.042630,-121.853970,2013-08-23 14:15:00,0.000333,0.008333,0.006778,58.0,72.5
5,1,40.498332,-122.535496,2013-09-09 12:32:00,0.000000,0.000000,0.025000,25.0,83.9
6,1,40.190062,-121.595555,2013-05-01 09:12:00,0.011000,0.018167,0.013778,15.0,76.0
7,1,34.786100,-118.941100,2013-05-15 12:50:00,0.014667,0.012000,0.023333,39.0,75.5
8,1,34.786100,-118.941100,2013-05-15 12:50:00,0.014667,0.012000,0.023333,39.0,75.5
9,1,38.816700,-122.849800,2013-11-22 02:04:00,0.034000,0.017000,0.011333,17.0,57.6


In [23]:
s3_output_path = "{}/{}/output".format(bucket, prefix)
train_data_path = "s3://{}/{}/train/train.csv".format(bucket, prefix)
test_data_path = "s3://{}/{}/test/test.json".format(bucket, prefix)

fire_data.to_csv(train_data_path, header=False, index=False)

# def series_to_obj(ls, cat=None):
#     obj = {"fire": ls[0], "lat": ls[1], "long":ls[2], "started": ls[3], "prec30": ls[4], "prec60":ls[5], "prec90":ls[6], "humidity":ls[7], "temp":ls[8]}
#     if cat:
#         obj["cat"] = cat
#     return obj


# def series_to_jsonline(ls, cat=None):
#     return json.dumps(series_to_obj(ls, cat))

# with open("train.json", "w") as fp:
#     for ls in listOfFire:
#         fp.write(series_to_jsonline(ls))
#         fp.write("\n")
# ! aws s3 mv train.json $train_data_path

# with open("test.json", "w") as fp:
#     for ts in violation_list:
#         fp.write(series_to_jsonline(ts))
#         fp.write("\n")
# ! aws s3 mv test.json $test_data_path

# Training Algorithm 

Copied from the AWS GitHub training repo, modified for our needs 

In [24]:
container = sagemaker.image_uris.retrieve("xgboost", sess.boto_region_name, "1.5-1")
# display(container)

s3_input_train = TrainingInput(
    s3_data="s3://{}/{}/train".format(bucket, prefix), content_type="csv"
)

In [26]:
# Hyperparameters

sess = sagemaker.Session()

xgb = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=1,
    instance_type="ml.m4.xlarge",
    output_path="s3://{}/{}/output".format(bucket, prefix),
    sagemaker_session=sess,
)
xgb.set_hyperparameters(
    max_depth=5,
    eta=0.2,
    gamma=4,
    min_child_weight=6,
    subsample=0.8,
    verbosity=0,
    objective="binary:logistic",
    num_round=100,
)
# This will take the two data sets and begin actual training of the model
xgb.fit({"train": s3_input_train})

# from sagemaker.tuner import (
#     IntegerParameter,
#     CategoricalParameter,
#     ContinuousParameter,
#     HyperparameterTuner,
# )
# from sagemaker import image_uris


# container = image_uris.retrieve(region=region, framework="forecasting-deepar")

# deepar = sagemaker.estimator.Estimator(
#     container,
#     role,
#     instance_count=1,
#     instance_type="ml.m4.xlarge",
#     use_spot_instances=True,  # use spot instances
#     max_run=1800,  # max training time in seconds
#     max_wait=1800,  # seconds to wait for spot instance
#     output_path="s3://{}/{}".format(bucket, s3_output_path),
#     sagemaker_session=sess,
# )
# freq = "D"
# context_length = 30

# deepar.set_hyperparameters(
#     time_freq=freq, context_length=str(context_length), prediction_length=str(33)
# )

# hyperparameter_ranges = {
#     "mini_batch_size": IntegerParameter(100, 400),
#     "epochs": IntegerParameter(200, 400),
#     "num_cells": IntegerParameter(30, 100),
#     "likelihood": CategoricalParameter(["negative-binomial", "student-T"]),
#     "learning_rate": ContinuousParameter(0.0001, 0.1),
# }

# objective_metric_name = "test:RMSE"

# tuner = HyperparameterTuner(
#     deepar,
#     objective_metric_name,
#     hyperparameter_ranges,
#     max_jobs=10,
#     strategy="Bayesian",
#     objective_type="Minimize",
#     max_parallel_jobs=10,
#     early_stopping_type="Auto",
# )

# s3_input_train = sagemaker.inputs.TrainingInput(
#     s3_data="s3://{}/{}/train/".format(bucket, prefix), content_type="json"
# )
# # s3_input_test = sagemaker.inputs.TrainingInput(
# #     s3_data="s3://{}/{}/test/".format(bucket, prefix), content_type="json"
# # )

# tuner.fit({"train": s3_input_train}, include_cls_metadata=False)
# tuner.wait()

2022-11-21 06:44:53 Starting - Starting the training job...
2022-11-21 06:45:21 Starting - Preparing the instances for trainingProfilerReport-1669013093: InProgress
............
2022-11-21 06:47:19 Downloading - Downloading input data......
2022-11-21 06:48:18 Training - Training image download completed. Training in progress.
2022-11-21 06:48:18 Uploading - Uploading generated training model[2022-11-21 06:48:08.506 ip-10-0-127-110.us-west-1.compute.internal:7 INFO utils.py:27] RULE_JOB_STOP_SIGNAL_FILENAME: None
[2022-11-21:06:48:08:INFO] Imported framework sagemaker_xgboost_container.training
[2022-11-21:06:48:08:INFO] Failed to parse hyperparameter objective value binary:logistic to Json.
Returning the value itself
[2022-11-21:06:48:08:INFO] No GPUs detected (normal if no gpus installed)
[2022-11-21:06:48:08:INFO] Running XGBoost Sagemaker in algorithm mode
[2022-11-21:06:48:08:INFO] Determined delimiter of CSV input is ','
[2022-11-21:06:48:08:INFO] files path: /opt/ml/input/data/t

# HOST

Now that we've trained the algorithm, let's create a model and deploy it to a hosted endpoint.

In [ ]:
xgb_predictor = xgb.deploy(
    initial_instance_count=1, instance_type="ml.m4.xlarge", serializer=CSVSerializer()
)

In [ ]:
xgb_predictor.delete_endpoint()